<a href="https://colab.research.google.com/github/patelami3431/Web-Crawler/blob/master/HLT_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import urllib
from urllib import request
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re

In [44]:
url = "https://www.medicalnewstoday.com/articles/5847.php"
p = request.urlopen(url).read()
soup = BeautifulSoup(p, 'html.parser')
# print(soup.get_text())

In [46]:
# row = soup.find(class_ = 'c-entry-content')
counter =0
s = []
for link in soup.find_all('a'):
#     if 'climatechange' in link.get('href'):
        s.append(link.get('href'))
        counter += 1
        
print(counter)

182


In [51]:
counter = 0
final = []
for link in s:
    if link and 'medical' in link:
        print(link)
        final.append(link)
        counter += 1
        
print(counter)

https://www.medicalnewstoday.com/privacy-policy#vendors
https://www.medicalnewstoday.com/privacy-policy
https://www.medicalnewstoday.com/advertising
https://www.medicalnewstoday.com/privacy-policy
http://www.medicalnewstoday.com/articles/7379.php
http://www.medicalnewstoday.com/articles/7382.php
http://www.medicalnewstoday.com/articles/180858.php
http://www.medicalnewstoday.com/articles/8749.php
http://www.medicalnewstoday.com/articles/149636.php
http://www.medicalnewstoday.com/articles/149454.php
http://www.medicalnewstoday.com/articles/7380.php
http://www.medicalnewstoday.com/articles/7381.php
http://www.medicalnewstoday.com/articles/149090.php
https://www.medicalnewstoday.com/articles/5847.php
https://www.pinterest.com/search/pins/?q=www.medicalnewstoday.com
https://www.medicalnewstoday.com/articles/5847.php
https://www.medicalnewstoday.com
17
